# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

In [8]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Inputs & Data

Explanation of each input and where it comes from.

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/nancy/user/riwata/projects/reward_comp_ext/results/2024_11_13_extract_lfp/proc/rce_pilot_2_both_rewarded_01_lfp_traces_and_frames.pkl")

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [10]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.1,1.1,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-16, -42, -198, -374, -340, -200, -169, -182,...","[315, 282, 139, -21, -38, 12, 25, 46, 91, 82, ...","[219, 203, 58, -151, -182, -60, 13, 38, 110, 1...","[-199, -485, -732, -759, -852, -811, -665, -49...","[-656, -620, -440, -71, 233, 207, 106, 219, 31..."
1,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.2,1.2,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[81, 226, 453, 521, 479, 528, 646, 664, 677, 7...","[119, 138, 229, 347, 383, 356, 301, 213, 222, ...","[96, 135, 227, 304, 317, 314, 313, 286, 306, 3...","[-101, -49, 36, 96, 160, 293, 445, 521, 614, 7...","[105, 146, 154, 15, -136, -147, -172, -393, -6..."
2,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.1,1.1,...,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-415, -550, -440, -166, -241, -480, -714, -86...","[-623, -905, -912, -711, -957, -1200, -1305, -...","[-431, -635, -620, -448, -625, -824, -954, -10...","[-332, -501, -580, -554, -701, -835, -945, -10...","[-421, -517, -517, -391, -340, -301, -339, -45..."
3,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.4,1.4,...,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[63, 92, 181, 225, 136, 16, -10, 52, 56, -52, ...","[-67, -55, 12, 98, 124, 96, 75, 108, 117, 49, ...","[-45, -30, 41, 133, 186, 178, 120, 88, 67, 13,...","[-30, 70, 192, 279, 410, 520, 590, 620, 640, 6...","[-205, -166, -122, -79, -1, 47, 51, 56, 35, -8..."
4,2,20230625_112913_standard_comp_to_both_rewarded...,"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2_both_rewarded"

In [12]:
FULL_LFP_TRACES_PKL = "{}_02_lfp_traces_and_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [13]:
ALL_PHY_DIR

['/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-1_t1b2L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce2/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/p

In [14]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [15]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,8,0,0,1,9.198713,3,0,60.0,0.217516,mua,746,0,9
1,9,1,1,2,12.508228,3,0,60.0,1.605419,good,5506,0,10
2,19,1,0,2,13.435120,15,0,300.0,0.503261,good,1726,0,20
3,20,0,0,0,7.342390,20,0,400.0,0.219265,mua,752,0,22
4,21,47,10,122,10.664793,21,0,420.0,0.605604,noise,2077,0,23


- Combining all the unit info dataframes and adding the recording name

In [16]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [17]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230616_111904_standard_comp_to_training_D4_s...,0,1,1,9,21.699682,0,0.0,0.0,1.868411,noise,6304,0,1.0
1,20230616_111904_standard_comp_to_training_D4_s...,6,19,12,34,8.349835,2,0.0,40.0,3.400128,mua,11472,0,7.0
2,20230616_111904_standard_comp_to_training_D4_s...,7,2,1,10,13.197960,2,0.0,40.0,11.119177,good,37516,0,8.0
3,20230616_111904_standard_comp_to_training_D4_s...,9,27,10,79,8.396154,3,0.0,60.0,5.852713,mua,19747,0,10.0
4,20230616_111904_standard_comp_to_training_D4_s...,10,8,1,27,7.519016,3,0.0,60.0,2.798170,mua,9441,0,11.0


- Filtering for the good units

In [18]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [19]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230616_111904_standard_comp_to_training_D4_s...,7,2,1,10,13.197960,2,0.0,40.0,11.119177,good,37516,0,8.0
1,20230616_111904_standard_comp_to_training_D4_s...,23,1,0,3,9.657406,13,0.0,260.0,1.115889,good,3765,0,25.0
2,20230616_111904_standard_comp_to_training_D4_s...,30,0,0,1,17.081881,15,0.0,300.0,0.904567,good,3052,0,32.0
3,20230616_111904_standard_comp_to_training_D4_s...,53,0,0,0,19.220345,24,0.0,480.0,2.608780,good,8802,0,47.0
4,20230616_111904_standard_comp_to_training_D4_s...,64,1,0,3,9.148628,20,0.0,400.0,1.644344,good,5548,0,39.0


In [20]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [21]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 82,  33, 121, ...,  36,  35,  36], dtype=int32)

In [23]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(829550,)

- The times that all the spikes happened

In [24]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[      93],
       [     151],
       [     335],
       ...,
       [67479567],
       [67479676],
       [67479729]])

In [26]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(829550, 1)

### Combining everything into a dataframe

In [27]:
recording_to_spike_df = {}

for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [28]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,15,0,0,1,14.411364,6,0.0,120.0,0.901093,good,3263,0,17.0
1,17,0,0,1,10.738522,8,0.0,160.0,1.143280,good,4140,0,19.0
2,19,0,0,0,11.480671,8,0.0,160.0,1.276939,noise,4624,0,21.0
3,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0
4,35,6,2,23,7.762705,19,0.0,380.0,2.981642,mua,10797,0,37.0


In [29]:
spike_times

array([[      89],
       [     116],
       [     188],
       ...,
       [72422846],
       [72422932],
       [72423021]])

In [30]:
spike_clusters

array([141, 254,  32, ..., 141, 157,  32], dtype=int32)

- Combining the spike time df for all recordings

In [31]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [32]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [33]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,23,416,23,1,0,3,9.657406,13,0.0,260.0,1.115889,good,3765,0,25.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
1,80,1123,80,7,3,11,7.514018,2,NaN,40.0,5.136944,good,17332,0,NaN,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
2,53,1395,53,0,0,0,19.220345,24,0.0,480.0,2.608780,good,8802,0,47.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
3,102,1830,102,0,0,0,17.968143,10,0.0,200.0,1.499115,good,5058,0,22.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN
4,30,2302,30,0,0,1,17.081881,15,0.0,300.0,0.904567,good,3052,0,32.0,20230616_111904_standard_comp_to_training_D4_s...,NaN,NaN


In [34]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
4630787,32,72421631,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,394.0,0.01970
4630788,32,72421974,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,343.0,0.01715
4630789,32,72422482,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,508.0,0.02540
4630790,141,72422846,141,29,15,62,7.976220,2,0.0,40.0,20.363095,good,73738,0,9.0,20230625_112913_standard_comp_to_both_rewarded...,3162.0,0.15810
4630791,32,72423021,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0,20230625_112913_standard_comp_to_both_rewarded...,539.0,0.02695


## Grouping all the neurons by recording

In [35]:
# Grouping all spike times by neuron and recording
grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: np.array(x)).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)

In [36]:
grouped_df.head()

,spike_clusters,recording_name,spike_times
0,13,20230616_111904_standard_comp_to_training_D4_s...,"[164503, 214157, 219948, 228496, 254211, 25605..."
1,43,20230616_111904_standard_comp_to_training_D4_s...,"[10518, 16788, 33219, 40252, 43753, 45536, 495..."
2,46,20230616_111904_standard_comp_to_training_D4_s...,"[79383, 181665, 237836, 301387, 327537, 351182..."
3,67,20230616_111904_standard_comp_to_training_D4_s...,"[71248, 72927, 78139, 79747, 80418, 81025, 813..."
4,68,20230616_111904_standard_comp_to_training_D4_s...,"[20982, 66688, 94142, 176325, 216673, 217233, ..."


In [37]:
max_number_of_spikes = all_spike_time_df["n_spikes"].max()

In [38]:
max_number_of_spikes

179245

In [39]:
grouped_df["spike_times"] = grouped_df["spike_times"].apply(lambda x: np.concatenate([x, np.full([max_number_of_spikes - x.shape[0]], np.nan)]))

In [40]:
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: np.array(list(x))}).reset_index()

In [41]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times
0,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.1,1.1,...,"[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-16, -42, -198, -374, -340, -200, -169, -182,...","[315, 282, 139, -21, -38, 12, 25, 46, 91, 82, ...","[219, 203, 58, -151, -182, -60, 13, 38, 110, 1...","[-199, -485, -732, -759, -852, -811, -665, -49...","[-656, -620, -440, -71, 233, 207, 106, 219, 31...",20230622_110832_standard_comp_to_both_rewarded...,"[1, 2, 15, 77, 110, 113, 115, 121, 148]","[[55448.0, 97552.0, 155723.0, 234331.0, 283736..."
1,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.2,1.2,...,"[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[81, 226, 453, 521, 479, 528, 646, 664, 677, 7...","[119, 138, 229, 347, 383, 356, 301, 213, 222, ...","[96, 135, 227, 304, 317, 314, 313, 286, 306, 3...","[-101, -49, 36, 96, 160, 293, 445, 521, 614, 7...","[105, 146, 154, 15, -136, -147, -172, -393, -6...",20230622_110832_standard_comp_to_both_rewarded...,"[20, 21, 22, 23, 39, 44, 46, 48, 58, 59, 146, ...","[[7114.0, 29239.0, 35085.0, 60921.0, 66984.0, ..."
2,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.1,1.1,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-415, -550, -440, -166, -241, -480, -714, -86...","[-623, -905, -912, -711, -957, -1200, -1305, -...","[-431, -635, -620, -448, -625, -824, -954, -10...","[-332, -501, -580, -554, -701, -835, -945, -10...","[-421, -517, -517, -391, -340, -301, -339, -45...",20230624_105855_standard_comp_to_both_rewarded...,"[0, 37, 39, 40, 42, 43, 44, 63, 113, 150, 158,...","[[15741.0, 21515.0, 25164.0, 27471.0, 40049.0,..."
3,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.4,1.4,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[63, 92, 181, 225, 136, 16, -10, 52, 56, -52, ...","[-67, -55, 12, 98, 124, 96, 75, 108, 117, 49, ...","[-45, -30, 41, 133, 186, 178, 120, 88, 67, 13,...","[-30, 70, 192, 279, 410, 520, 590, 620, 640, 6...","[-205, -166, -122, -79, -1, 47, 51, 56, 35, -8...",20230624_105855_standard_comp_to_both_rewarded...,"[133, 150, 152, 154, 173, 176, 177]","[[128399.0, 182521.0, 183083.0, 195446.0, 1997..."
4,2,20230625_112913_standard_comp_to_both_rewarded...,"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.1,1.1,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 1

## Calculating the firing rates

In [43]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'recording_name', 'spike_clusters',
       'spike_times'],
      dtype='object')

In [44]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,15,0,0,1,14.411364,6,0.0,120.0,0.901093,good,3263,0,17.0
1,17,0,0,1,10.738522,8,0.0,160.0,1.143280,good,4140,0,19.0
2,19,0,0,0,11.480671,8,0.0,160.0,1.276939,noise,4624,0,21.0
3,32,110,43,323,8.556771,17,0.0,340.0,41.800155,good,151365,0,34.0
4,35,6,2,23,7.762705,19,0.0,380.0,2.981642,mua,10797,0,37.0


In [45]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times
0,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.1,1.1,...,"[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-16, -42, -198, -374, -340, -200, -169, -182,...","[315, 282, 139, -21, -38, 12, 25, 46, 91, 82, ...","[219, 203, 58, -151, -182, -60, 13, 38, 110, 1...","[-199, -485, -732, -759, -852, -811, -665, -49...","[-656, -620, -440, -71, 233, 207, 106, 219, 31...",20230622_110832_standard_comp_to_both_rewarded...,"[1, 2, 15, 77, 110, 113, 115, 121, 148]","[[55448.0, 97552.0, 155723.0, 234331.0, 283736..."
1,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.2,1.2,...,"[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[81, 226, 453, 521, 479, 528, 646, 664, 677, 7...","[119, 138, 229, 347, 383, 356, 301, 213, 222, ...","[96, 135, 227, 304, 317, 314, 313, 286, 306, 3...","[-101, -49, 36, 96, 160, 293, 445, 521, 614, 7...","[105, 146, 154, 15, -136, -147, -172, -393, -6...",20230622_110832_standard_comp_to_both_rewarded...,"[20, 21, 22, 23, 39, 44, 46, 48, 58, 59, 146, ...","[[7114.0, 29239.0, 35085.0, 60921.0, 66984.0, ..."
2,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.1,1.1,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-415, -550, -440, -166, -241, -480, -714, -86...","[-623, -905, -912, -711, -957, -1200, -1305, -...","[-431, -635, -620, -448, -625, -824, -954, -10...","[-332, -501, -580, -554, -701, -835, -945, -10...","[-421, -517, -517, -391, -340, -301, -339, -45...",20230624_105855_standard_comp_to_both_rewarded...,"[0, 37, 39, 40, 42, 43, 44, 63, 113, 150, 158,...","[[15741.0, 21515.0, 25164.0, 27471.0, 40049.0,..."
3,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.4,1.4,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[63, 92, 181, 225, 136, 16, -10, 52, 56, -52, ...","[-67, -55, 12, 98, 124, 96, 75, 108, 117, 49, ...","[-45, -30, 41, 133, 186, 178, 120, 88, 67, 13,...","[-30, 70, 192, 279, 410, 520, 590, 620, 640, 6...","[-205, -166, -122, -79, -1, 47, 51, 56, 35, -8...",20230624_105855_standard_comp_to_both_rewarded...,"[133, 150, 152, 154, 173, 176, 177]","[[128399.0, 182521.0, 183083.0, 195446.0, 1997..."
4,2,20230625_112913_standard_comp_to_both_rewarded...,"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.1,1.1,...,[],"[0, 20, 40, 60, 80, 100, 120, 140, 1

In [46]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times[~np.isnan(times)]), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)


In [47]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0][~np.isnan(x["spike_times"][0])], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [48]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [49]:
LFP_SPECTRAL_DF.iloc[:5,:10]

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject
0,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.1,1.1
1,2,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.2,1.2
2,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.1,1.1
3,2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.4,1.4
4,2,20230625_112913_standard_comp_to_both_rewarded...,"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.1,1.1


In [50]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace
0,"[1.1, 1.2]",4537337,74904369,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-16, -42, -198, -374, -340, -200, -169, -182,...","[315, 282, 139, -21, -38, 12, 25, 46, 91, 82, ..."
1,"[1.1, 1.2]",4537337,76591466,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[81, 226, 453, 521, 479, 528, 646, 664, 677, 7...","[119, 138, 229, 347, 383, 356, 301, 213, 222, ..."
2,"[1.2, 1.4]",8948560,80713042,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-415, -550, -440, -166, -241, -480, -714, -86...","[-623, -905, -912, -711, -957, -1200, -1305, -..."
3,"[1.2, 1.4]",8948560,80713042,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[63, 92, 181, 225, 136, 16, -10, 52, 56, -52, ...","[-67, -55, 12, 98, 124, 96, 75, 108, 117, 49, ..."
4,"[1.1, 1.4]",6771359,79195515,"[1384, 2770, 2770, 4156, 5542, 6928, 6928, 831...","[[1103260, 1303263], [3503289, 3703294], [5803...","[[310252, 314054], [1167262, 1196060], [156166...",[],"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[411, 420, 387, 334, 292, 219, 240, 426, 493, ...","[590, 499, 395, 400, 436, 464, 485, 488, 452, ..."


In [51]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[219, 203, 58, -151, -182, -60, 13, 38, 110, 1...","[-199, -485, -732, -759, -852, -811, -665, -49...","[-656, -620, -440, -71, 233, 207, 106, 219, 31...",20230622_110832_standard_comp_to_both_rewarded...,"[1, 2, 15, 77, 110, 113, 115, 121, 148]","[[55448.0, 97552.0, 155723.0, 234331.0, 283736...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[96, 135, 227, 304, 317, 314, 313, 286, 306, 3...","[-101, -49, 36, 96, 160, 293, 445, 521, 614, 7...","[105, 146, 154, 15, -136, -147, -172, -393, -6...",20230622_110832_standard_comp_to_both_rewarded...,"[20, 21, 22, 23, 39, 44, 46, 48, 58, 59, 146, ...","[[7114.0, 29239.0, 35085.0, 60921.0, 66984.0, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,"[-431, -635, -620, -448, -625, -824, -954, -10...","[-332, -501, -580, -554, -701, -835, -945, -10...","[-421, -517, -517, -391, -340, -301, -339, -45...",20230624_105855_standard_comp_to_both_rewarded...,"[0, 37, 39, 40, 42, 43, 44, 63, 113, 150, 158,...","[[15741.0, 21515.0, 25164.0, 27471.0, 40049.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,"[-45, -30, 41, 133, 186, 178, 120, 88, 67, 13,...","[-30, 70, 192, 279, 410, 520, 590, 620, 640, 6...","[-205, -166, -122, -79, -1, 47, 51, 56, 35, -8...",20230624_105855_standard_comp_to_both_rewarded...,"[133, 150, 152, 154, 173, 176, 177]","[[128399.0, 182521.0, 183083.0, 195446.0, 1997...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
4,"[432, 417, 412, 454, 512, 535, 500, 420, 340, ...","[400, 305, 262, 342, 330, 346, 557, 762, 726, ...","[516, 466, 553, 779, 868, 856, 777, 587, 355, ...",20230625_112913_standard_comp_to_both_rewarded...,"[0, 7, 11, 13, 27, 52, 64, 67, 68, 105, 108, 1...","[[407.0, 2413.0, 3667.0, 5935.0, 6395.0, 7088....","[[1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0, 2.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."


In [52]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

""
0
1
2
3
4


In [53]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

(35183,)

In [54]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

(9, 35183)

In [55]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

11.0

In [56]:
LFP_SPECTRAL_DF.to_pickle(os.path.join("./proc", FULL_LFP_TRACES_PKL))

In [57]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'recording_name', 'spike_clusters',
       'spike_times', 'neuron_average_fr', 'neuron_average_timestamps'],
      dtype='object')

In [58]:
LFP_SPECTRAL_DF["spike_clusters"].iloc[0]

[1, 2, 15, 77, 110, 113, 115, 121, 148]

In [59]:
LFP_SPECTRAL_DF["spike_times"].iloc[0].shape

(9, 179245)

In [60]:
raise ValueError("")

ValueError: 